# Implementation Quantum Teleportation algorithm in python 
----------

In [1]:
pip install qiskit


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install qiskit-aer


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pylatexenc


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install qiskit-ibm-runtime # install the missing module
# Import necessary libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram, circuit_drawer

import numpy as np
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler # this should now work
QiskitRuntimeService.save_account(token='141275b0005f54bb748ec04ba4a27e703803a4a1343847ba4eb8a21674fdb9359dae0f65ea378e883dad6756e4ec46c919e6eaf22c17fd08aa699acb0a6cea36',
                                  channel='ibm_quantum',
                                  overwrite = True)

# Initialize IBM Quantum account

service = QiskitRuntimeService()


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# Import necessary libraries
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram
from qiskit_aer import Aer
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
import matplotlib.pyplot as plt

# Initialize IBM Quantum account
service = QiskitRuntimeService(channel="ibm_quantum")

# Create the quantum circuit with 3 qubits and 3 classical bits
q = QuantumRegister(3, 'q')  # Quantum register
c0 = ClassicalRegister(1, 'c0')  # Classical register for Alice's measurement
c1 = ClassicalRegister(1, 'c1')  # Classical register for Bob's measurement
c2 = ClassicalRegister(1, 'c2')  # Classical register for the final state
circuit = QuantumCircuit(q, c0, c1, c2)

# Prepare the initial state to be teleported
circuit.initialize([0, 1], q[0])  # Apply X gate to put in state |1>
circuit.barrier()


# Create an entanglement between Alice's and Bob's qubits
circuit.h(q[1])        # Hadamard on q[1]
circuit.cx(q[1], q[2])  # CNOT gate to entangle q[1] and q[2]
circuit.barrier()

# Teleportation process
circuit.cx(q[0], q[1])  # CNOT between q[0] and q[1]
circuit.h(q[0])         # Hadamard on q[0]
circuit.barrier()

# Measure Alice's qubits and send the measurement results to Bob
circuit.measure(q[0], c0[0])  # Measure q[0] into c0
circuit.measure(q[1], c1[0])  # Measure q[1] into c1

# Apply corrective operations on Bob's qubit based on the measurement results
circuit.x(q[2]).c_if(c1, 1)  # Apply X if Alice's second qubit (c1) is 1
circuit.z(q[2]).c_if(c0, 1)  # Apply Z if Alice's first qubit (c0) is 1

# Measure the teleported qubit
circuit.measure(q[2], c2[0])  # Measure Bob's qubit into c2

# Visualize the circuit
print(circuit)
circuit_drawer(circuit, output='mpl')

# Simulate the circuit using the QASM simulator
simulator = Aer.get_backend('qasm_simulator')
job = simulator.run(circuit, shots=1)  # Run the circuit on the QASM simulator
result = job.result()
teleported_state = result.get_counts(circuit)  # Get the counts of the measurement results

# Print the teleported state
print("Teleported state:", teleported_state)


      ┌─────────────────┐ ░            ░      ┌───┐ ░ ┌─┐                    
 q_0: ┤ Initialize(0,1) ├─░────────────░───■──┤ H ├─░─┤M├────────────────────
      └─────────────────┘ ░ ┌───┐      ░ ┌─┴─┐└───┘ ░ └╥┘┌─┐                 
 q_1: ────────────────────░─┤ H ├──■───░─┤ X ├──────░──╫─┤M├─────────────────
                          ░ └───┘┌─┴─┐ ░ └───┘      ░  ║ └╥┘ ┌───┐  ┌───┐ ┌─┐
 q_2: ────────────────────░──────┤ X ├─░────────────░──╫──╫──┤ X ├──┤ Z ├─┤M├
                          ░      └───┘ ░            ░  ║  ║  └─╥─┘  └─╥─┘ └╥┘
                                                       ║  ║    ║   ┌──╨──┐ ║ 
c0: 1/═════════════════════════════════════════════════╩══╬════╬═══╡ 0x1 ╞═╬═
                                                       0  ║ ┌──╨──┐└─────┘ ║ 
c1: 1/════════════════════════════════════════════════════╩═╡ 0x1 ╞════════╬═
                                                          0 └─────┘        ║ 
c2: 1/══════════════════════════════════════════════════════════

In [6]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.library import QFT
import numpy as np

# Define pi from numpy
pi = np.pi

# Define Quantum and Classical Registers
q = QuantumRegister(5, 'q')
c = ClassicalRegister(5, 'c')

# Create Quantum Circuit
circuit = QuantumCircuit(q, c)

# Apply X gates on specific qubits
circuit.x(q[4])
circuit.x(q[2])
circuit.x(q[0])

# Apply the Quantum Fourier Transform (QFT)
circuit.append(QFT(num_qubits=5), q)

# Measure the qubits
circuit.measure(q, c)

# Draw the circuit
circuit.draw(output='mpl', filename='qft1.png')

# Print the circuit to console
print(circuit)


     ┌───┐┌──────┐┌─┐            
q_0: ┤ X ├┤0     ├┤M├────────────
     └───┘│      │└╥┘┌─┐         
q_1: ─────┤1     ├─╫─┤M├─────────
     ┌───┐│      │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 QFT ├─╫──╫─┤M├──────
     └───┘│      │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├─╫──╫──╫─┤M├───
     ┌───┐│      │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ X ├┤4     ├─╫──╫──╫──╫─┤M├
     └───┘└──────┘ ║  ║  ║  ║ └╥┘
c: 5/══════════════╩══╩══╩══╩══╩═
                   0  1  2  3  4 


In [7]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.circuit.library import QFT
import numpy as np

# Define pi from numpy
pi = np.pi

# First circuit diagram
q1 = QuantumRegister(5, 'q')
c1 = ClassicalRegister(5, 'c')
circuit1 = QuantumCircuit(q1, c1)

circuit1.x(q1[4])
circuit1.x(q1[2])
circuit1.x(q1[0])
circuit1.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=False, insert_barriers=False, name='qft'), q1)
circuit1.measure(q1, c1)

# Draw the first circuit and save it as 'qft1.png'
circuit1.draw(output='mpl', filename='qft1.png')

# Print the first circuit
print(circuit1)


# Second circuit diagram
q2 = QuantumRegister(5, 'q')
c2 = ClassicalRegister(5, 'c')
circuit2 = QuantumCircuit(q2, c2)

circuit2.x(q2[4])
circuit2.x(q2[2])
circuit2.x(q2[0])
circuit2.append(QFT(num_qubits=5, approximation_degree=0, do_swaps=True, inverse=False, insert_barriers=False, name='qft'), q2)
circuit2.measure(q2, c2)

# Draw the second circuit and save it as 'qft2.png'
circuit2.draw(output='mpl', filename='qft2.png')

# Print the second circuit
print(circuit2)


     ┌───┐┌──────┐┌─┐            
q_0: ┤ X ├┤0     ├┤M├────────────
     └───┘│      │└╥┘┌─┐         
q_1: ─────┤1     ├─╫─┤M├─────────
     ┌───┐│      │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 qft ├─╫──╫─┤M├──────
     └───┘│      │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├─╫──╫──╫─┤M├───
     ┌───┐│      │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ X ├┤4     ├─╫──╫──╫──╫─┤M├
     └───┘└──────┘ ║  ║  ║  ║ └╥┘
c: 5/══════════════╩══╩══╩══╩══╩═
                   0  1  2  3  4 
     ┌───┐┌──────┐┌─┐            
q_0: ┤ X ├┤0     ├┤M├────────────
     └───┘│      │└╥┘┌─┐         
q_1: ─────┤1     ├─╫─┤M├─────────
     ┌───┐│      │ ║ └╥┘┌─┐      
q_2: ┤ X ├┤2 qft ├─╫──╫─┤M├──────
     └───┘│      │ ║  ║ └╥┘┌─┐   
q_3: ─────┤3     ├─╫──╫──╫─┤M├───
     ┌───┐│      │ ║  ║  ║ └╥┘┌─┐
q_4: ┤ X ├┤4     ├─╫──╫──╫──╫─┤M├
     └───┘└──────┘ ║  ║  ║  ║ └╥┘
c: 5/══════════════╩══╩══╩══╩══╩═
                   0  1  2  3  4 
